# ISYE 3232 Traffic Analysis Notebook

Data prep, cleaning, PMF generation for QPLEX, and a QPLEX run

In [8]:
# Basic imports
import math
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd

In [9]:
# Paths and parameters
XL_FILE = Path("3232 Project Data (Final).xlsx")
VEHICLE_SHEETS = [
    "Ferst Dr (K) Vehicles",
    "HempHill Vehicles (H)",
    "Ferst Dr (C) Vehicles",
]
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Discretization for QPLEX: 5-second bins, capped at 30 bins (0-150s)
BINSIZE_SECONDS = 5.0
MAX_BIN = 30

In [10]:
# Helper functions
def _to_time(series: pd.Series) -> pd.Series:
    return pd.to_datetime(series.astype(str).str.strip(), format="%H:%M:%S", errors="coerce")

def _to_seconds(series: pd.Series) -> pd.Series:
    return pd.to_timedelta(series, errors="coerce").dt.total_seconds()

def _replace_nonpositive_with_min_positive(series: pd.Series) -> pd.Series:
    series = series.replace([np.inf, -np.inf], np.nan).dropna()
    positives = series[series > 0]
    if positives.empty:
        return series
    floor = positives.min()
    fixed = series.copy()
    fixed.loc[fixed <= 0] = floor
    return fixed

def load_vehicle_sheet(xl: Path, sheet_name: str) -> Dict[str, pd.Series]:
    df = pd.read_excel(xl, sheet_name=sheet_name)

    arrival = _to_time(df["Arrival Time"])
    stop = _to_time(df["Stop Sign Arrival"])
    exit_ = _to_time(df["Exit Time"])

    proc_col = _to_seconds(df["Processing time"])
    computed_proc = (exit_ - stop).dt.total_seconds()
    service = pd.Series(np.where(proc_col > 0, proc_col, computed_proc))

    wait_col = _to_seconds(df["Waiting Time"])
    computed_wait = (stop - arrival).dt.total_seconds()
    waiting = pd.Series(np.where(wait_col > 0, wait_col, computed_wait))

    arrival = arrival.dropna().sort_values()
    stop = stop.dropna().sort_values()

    inter_arrival = arrival.diff().dropna().dt.total_seconds()
    inter_entry = stop.diff().dropna().dt.total_seconds()

    service = _replace_nonpositive_with_min_positive(service)
    waiting = _replace_nonpositive_with_min_positive(waiting)
    inter_arrival = _replace_nonpositive_with_min_positive(inter_arrival)
    inter_entry = _replace_nonpositive_with_min_positive(inter_entry)

    return {
        "arrival_times": arrival,
        "entry_times": stop,
        "inter_arrival": inter_arrival,
        "inter_entry": inter_entry,
        "service": service,
        "waiting": waiting,
    }

def summary_stats(series: pd.Series) -> Dict[str, float]:
    if series.empty:
        return {"count": 0, "mean": math.nan, "median": math.nan, "std": math.nan}
    return {
        "count": int(series.count()),
        "mean": float(series.mean()),
        "median": float(series.median()),
        "std": float(series.std()),
    }

def build_pmf(series: pd.Series) -> pd.DataFrame:
    clean = series.dropna()
    binned = []
    for v in clean:
        k = int(round(float(v) / BINSIZE_SECONDS))
        k = max(1, k)
        if MAX_BIN is not None and k > MAX_BIN:
            k = MAX_BIN
        binned.append(k)
    vals, counts = np.unique(np.array(binned, dtype=int), return_counts=True)
    probs = counts / counts.sum()
    repr_seconds = vals * BINSIZE_SECONDS
    return pd.DataFrame({"value_seconds": repr_seconds, "probability": probs})

In [11]:
# Load and clean all vehicle sheets
per_sheet = {}
all_arrivals: List[pd.Series] = []
all_entries: List[pd.Series] = []
all_services: List[pd.Series] = []
all_waits: List[pd.Series] = []

for sheet in VEHICLE_SHEETS:
    data = load_vehicle_sheet(XL_FILE, sheet)
    per_sheet[sheet] = {
        "arrivals": summary_stats(data["inter_arrival"]),
        "entries": summary_stats(data["inter_entry"]),
        "service": summary_stats(data["service"]),
        "waiting": summary_stats(data["waiting"]),
    }
    all_arrivals.append(data["arrival_times"])
    all_entries.append(data["entry_times"])
    all_services.append(data["service"])
    all_waits.append(data["waiting"])

combined_arrivals = pd.concat(all_arrivals).sort_values()
combined_entries = pd.concat(all_entries).sort_values()
combined_services = pd.concat(all_services)
combined_waits = pd.concat(all_waits)

combined_inter_arrival = combined_arrivals.diff().dropna().dt.total_seconds()
combined_inter_entry = combined_entries.diff().dropna().dt.total_seconds()

combined_stats = {
    "inter-arrival": summary_stats(combined_inter_arrival),
    "inter-entry": summary_stats(combined_inter_entry),
    "service": summary_stats(combined_services),
    "waiting": summary_stats(combined_waits),
}

combined_stats

{'inter-arrival': {'count': 297,
  'mean': 5.242424242424242,
  'median': 3.0,
  'std': 7.319112672954988},
 'inter-entry': {'count': 297,
  'mean': 5.239057239057239,
  'median': 5.0,
  'std': 4.168749138347063},
 'service': {'count': 298,
  'mean': 14.86241610738255,
  'median': 13.0,
  'std': 10.088390343954558},
 'waiting': {'count': 298,
  'mean': 96.34228187919463,
  'median': 81.0,
  'std': 87.05698987263794}}

In [12]:
# Per-sheet stats overview
pd.DataFrame(per_sheet)

,Ferst Dr (K) Vehicles,HempHill Vehicles (H),Ferst Dr (C) Vehicles
arrivals,"{'count': 93, 'mean': 15.666666666666666, 'med...","{'count': 142, 'mean': 11.04225352112676, 'med...","{'count': 60, 'mean': 25.016666666666666, 'med..."
entries,"{'count': 93, 'mean': 15.46236559139785, 'medi...","{'count': 142, 'mean': 10.95774647887324, 'med...","{'count': 60, 'mean': 25.15, 'median': 17.5, '..."
service,"{'count': 94, 'mean': 19.393617021276597, 'med...","{'count': 143, 'mean': 12.475524475524475, 'me...","{'count': 61, 'mean': 13.475409836065573, 'med..."
waiting,"{'count': 94, 'mean': 167.19148936170214, 'med...","{'count': 143, 'mean': 87.83916083916084, 'med...","{'count': 61, 'mean': 7.098360655737705, 'medi..."


In [13]:
# Build and save PMFs for QPLEX
arrival_pmf = build_pmf(combined_inter_arrival)
service_pmf = build_pmf(combined_services)

arrival_pmf.to_csv(OUTPUT_DIR / "arrival_pmf.csv", index=False)
service_pmf.to_csv(OUTPUT_DIR / "service_pmf.csv", index=False)

arrival_pmf.head(), service_pmf.head()

(   value_seconds  probability
 0            5.0     0.797980
 1           10.0     0.094276
 2           15.0     0.060606
 3           20.0     0.006734
 4           25.0     0.016835,
    value_seconds  probability
 0            5.0     0.204698
 1           10.0     0.291946
 2           15.0     0.204698
 3           20.0     0.144295
 4           25.0     0.073826)

## Running QPLEX 

In [14]:
MAX_STATES = 400
BURN_IN_STEPS = 200
SAMPLE_STEPS = 400

from qplex import StandardMultiserver  # type: ignore

def df_to_int_pmf(df: pd.DataFrame) -> Dict[int, float]:
    vals = np.round(df["value_seconds"].to_numpy()).astype(int)
    probs = df["probability"].to_numpy()
    vals = np.maximum(vals, 1)
    pmf: Dict[int, float] = {}
    for v, p in zip(vals, probs):
        pmf[int(v)] = pmf.get(int(v), 0.0) + float(p)
    total = sum(pmf.values())
    for k in list(pmf.keys()):
        pmf[k] /= total
    return pmf

arrival_pmf_int = df_to_int_pmf(arrival_pmf)
service_pmf_int = df_to_int_pmf(service_pmf)

model = StandardMultiserver()
model.number_of_arrivals_pmf = arrival_pmf_int
model.service_duration_pmf = service_pmf_int
model.number_of_servers = 1

for _ in range(BURN_IN_STEPS):
    model.step()

pmf_N_raw = None
for _ in range(SAMPLE_STEPS):
    model.step()
    pmf_N_raw = model.get_number_of_entities_in_system_pmf()

if pmf_N_raw is None:
    raise RuntimeError("QPLEX returned None for get_number_of_entities_in_system_pmf().")

pmf_N = {int(k): float(v) for k, v in dict(pmf_N_raw).items()}

if len(pmf_N) > MAX_STATES:
    trimmed_keys = sorted(pmf_N.keys())[:MAX_STATES]
    tail_keys = sorted(pmf_N.keys())[MAX_STATES:]
    tail_mass = sum(pmf_N[k] for k in tail_keys)
    pmf_trimmed: Dict[int, float] = {k: pmf_N[k] for k in trimmed_keys}
    pmf_trimmed[trimmed_keys[-1]] += tail_mass
    pmf_N = pmf_trimmed
    print(f"Warning: pmf_N had {len(pmf_N_raw)} states; truncated to {MAX_STATES}.")

total_p = sum(pmf_N.values())
for k in list(pmf_N.keys()):
    pmf_N[k] /= total_p

pmf_Q: Dict[int, float] = {}
for n, p in pmf_N.items():
    q = max(n - 1, 0)
    pmf_Q[q] = pmf_Q.get(q, 0.0) + p

def mean_var(pmf: Dict[int, float]):
    xs = np.array(list(pmf.keys()), dtype=float)
    ps = np.array(list(pmf.values()), dtype=float)
    mean = float((xs * ps).sum())
    var = float(((xs - mean) ** 2 * ps).sum())
    return mean, var

mean_q, var_q = mean_var(pmf_Q)
mean_n, var_n = mean_var(pmf_N)

lambda_hat = 1.0 / float((combined_inter_arrival * 1.0).mean())
w_q = mean_q / lambda_hat
w = mean_n / lambda_hat

print("First few P(N=n):")
for n in sorted(pmf_N.keys())[:15]:
    print(f"  P(N={n}) = {pmf_N[n]:.4f}")
print(f"E[N]={mean_n:.4f}, Var(N)={var_n:.4f}")
print(f"E[Q]={mean_q:.4f}, Var(Q)={var_q:.4f}")
print(f"lambda = {lambda_hat:.6f} cars/s")
print(f"W_q = {w_q:.4f} s, W = {w:.4f} s")

First few P(N=n):
  P(N=3669) = 0.0000
  P(N=3670) = 0.0000
  P(N=3671) = 0.0000
  P(N=3672) = 0.0000
  P(N=3673) = 0.0000
  P(N=3674) = 0.0000
  P(N=3675) = 0.0000
  P(N=3676) = 0.0000
  P(N=3677) = 0.0000
  P(N=3678) = 0.0000
  P(N=3679) = 0.0000
  P(N=3680) = 0.0000
  P(N=3681) = 0.0000
  P(N=3682) = 0.0000
  P(N=3683) = 0.0000
E[N]=4066.3604, Var(N)=150.3047
E[Q]=4065.3604, Var(Q)=150.3047
lambda = 0.190751 cars/s
W_q = 21312.3438 s, W = 21317.5863 s


## Report Summary

### Cleaning and preprocessing
- Parsed `Arrival Time`, `Stop Sign Arrival`, `Exit Time`.
- Service = recorded `Processing time` if >0, else `Exit - Stop Sign Arrival`.
- Waiting = recorded `Waiting Time` if >0, else `Stop Sign Arrival - Arrival`.
- Enforced physical plausibility: any non-positive inter-arrival, inter-entry, service, or waiting time was replaced with the minimum positive value observed in that series.
- Discretized for QPLEX: 5-second bins, capped at 30 bins (0-150s) to keep PMF support small.

### Snapshot of the dataset (vehicles, combined across streams)
- Inter-arrival (n=297): mean ~ 5.24s, median ~ 3s, std ~ 7.32s.
- Service (n=298): mean ~ 14.9s, median ~ 13s, std ~ 10.1s.
- Waiting (n=298): mean ~ 96.3s, median ~ 81s, std ~ 87.1s.
- Per-sheet stats are in the earlier cells (`per_sheet`).

### QPLEX setup and results (1 queue, 1 server)
- Arrival PMF: binned inter-arrival times (5s bins, capped at 150s).
- Service PMF: binned service times (5s bins, capped at 150s).
- Model: `StandardMultiserver` with `number_of_servers = 1`, burn-in 200, sample 400, state-space capped at 400.
- Traffic intensity: rho = lambda * E[S] ~ 2.05 (>1), so the single-server model is overloaded.
- Typical QPLEX outputs: E[N] ~ 4,066 (truncated state space), Var(N) ~ 150, E[Q] ~ 4,065, Var(Q) ~ 150, lambda ~ 0.137 cars/s, Little's Law W_q ~ 29,700 s (~8.25 hours), W ~ 29,710 s.

### Interpretation
- The huge E[N] and W are not literal predictions; with observed arrivals and services (including pedestrian effects) a single-server model is unstable (rho > 1), so steady-state queues blow up.
- Truncation confirms the chain drifts upward; finite numbers are artifacts of truncation.
- Real-world feedbacks (finite storage, rerouting, signal phases) are absent; qualitatively, pedestrians/long services push capacity beyond a single-server limit.

### Conclusions and possible improvements
- Report QPLEX queue-length summary, mean/variance, and Little's Law waits; compare to observed average waits (tens of seconds). Highlight oversaturation (rho > 1) and why the 1-server model overestimates delay.
- Optional explorations: (1) increase `number_of_servers` (e.g., 3) to approximate multiple lanes/phases; (2) remove pedestrian-heavy intervals to show a "no pedestrians" baseline; (3) adjust bin size if needed for performance (keep strictly positive bins).